In [1]:
%config Completer.use_jedi = False #for auto complete code 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import skew
#%matplotlib inline
plt.style.use('ggplot')

#funcion parachekar si los eneros estan completos
def mescompleto(x,ano):
    a=x[x.index.year==ano].isnull().sum()==0
    return a
#retorna años con los meses completos
def anosconmesescompletos(x):
    ano=range(1981,2017)
    idx=[]
    for i in range(len(ano)):
        if mescompleto(x,ano[i]):
            idx.append(ano[i])
    return idx

#retorna la serie con los meses completos para luego calcular los estadisticos
def seriefinal(x):

    idx=anosconmesescompletos(x)
    lista=[]
    for i in range(len(idx)):
        lista.append(x[x.index.year==idx[i]])
    
    if len(lista)>15:#mayor a 15 ano
        return pd.concat(lista)#remodificar esta funcion
    else:
        return []

def proportiondry(x):
    def mayorque(y):
        return y>0.1
    drydays=len(x)-np.sum(x.apply(mayorque))
    return float(drydays)/len(x)

def autocovariance(Xi, k=1):
    N = np.size(Xi)
    Xs = np.average(Xi)
    autoCov = 0
    for i in np.arange(0, N-k):
        autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
    return (1.0/(N-1))*autoCov

def estadisticosprec(x):
    result=[]
    a=x.dropna()
    #si se tiene menor a 10 años no contar
    if len(a)==0:
        return [None]
    result.append(np.mean(a))
    result.append(np.var(a))
    result.append(autocovariance(a))
    result.append(proportiondry(a))
    return result

#crea una tabla de los estadisticos extraidos una vez usado la fucion estadisticosprec
def crearTablaEst(location,x,names=['x','y','mean1h','var1h','autocov1h','dryperiod1h']):
    
    val=x.transpose()
    val.reset_index(drop=True, inplace=True)
    pos=location#posiciones.loc[final_station,:]
    pos.reset_index(drop=True, inplace=True)
    df=pd.concat([pos, pd.DataFrame(np.matrix(val))], axis=1)
    
    df.columns=names
    
    return df

def factores(a,b):
    if a==0.0 or b==0.0:
        return a
    else:
        return float(a)/b

In [15]:
def csv_trmm(file,month,month_hourly_date):
    month_sat_hourly=pd.read_csv(file)
    rng = month_hourly_date #bug generator
    month_hourly_date=rng[np.in1d(rng.month, month)]
    location=month_sat_hourly.iloc[:,0:2]
    location.columns=['x','y']
    month_sat_hourly=month_sat_hourly.iloc[:, 2:month_sat_hourly.shape[1]].transpose() # possible bug
    month_sat_hourly=month_sat_hourly.set_index(month_hourly_date)
    return (month_sat_hourly,location,month_hourly_date)

def common_period(dates,month,obs,daily_sat):
    daily_dates=dates[np.in1d(dates.month, month)]
    month_sat_daily=daily_sat.loc[daily_dates]
    month_obs_daily=obs.loc[daily_dates]
    
    return (month_sat_daily,month_obs_daily,daily_dates)

def filter_months_complete(percentaje,month_obs_daily,month_sat_daily,gauge_loc,month_sat_hourly):
    idx=[]
    n=percentaje*month_obs_daily.shape[0]
    
    for x in month_obs_daily.columns:
        if month_obs_daily[x].count()> n:
            idx.append(x)
            
    gauge_loc=gauge_loc.iloc[idx,:] 
    month_sat_daily=month_sat_daily.iloc[:,idx]
    month_obs_daily=month_obs_daily.iloc[:,idx]
    month_sat_hourly=month_sat_hourly.iloc[:,idx]
    
    return (gauge_loc,month_sat_daily,month_obs_daily,month_sat_hourly)   

def bias_correction(month_sat_daily,month_obs_daily,month_dates,month_sat_hourly,daily_dates):
    BIASM=[]
    for i in range(0,month_sat_daily.shape[0]):#filas
        BIAS=[]
        for j in range(0,month_sat_daily.shape[1]):#columnas
            BIAS.append(factores(month_obs_daily.iloc[i,j],month_sat_daily.iloc[i,j]))
        BIASM.append(BIAS)

        correcionM=[]
    bugM=[]
    k=0 #lleva la contabilidad de las fechas diarias
    for i in range(0,month_sat_hourly.shape[0]):
        correcion=[]
        bug=[]
        for j in range(0,month_sat_hourly.shape[1]):
            if month_dates[i].date()==daily_dates[k].date():

                bug.append(BIASM[k][j])
                correcion.append(month_sat_hourly.iloc[i,j]*BIASM[k][j])
                k=k
            else:
                k=k+1
                correcion.append(month_sat_hourly.iloc[i,j]*BIASM[k][j])

        bugM.append(bug)
        correcionM.append(correcion)
    
    month_sat_hourly_corrected=pd.DataFrame(correcionM)
    month_sat_hourly_corrected=month_sat_hourly_corrected.set_index(month_dates)
    
    return month_sat_hourly_corrected

def stats_month(month_sat_hourly_corrected,month_obs_daily,location):
    DATA3=month_sat_hourly_corrected.groupby(pd.Grouper(freq='3h')).agg(np.sum)
    EST3=DATA3.loc[np.in1d(DATA3.index.month, 2)].dropna().apply(estadisticosprec, axis=0)
    DATA6=month_sat_hourly_corrected.groupby(pd.Grouper(freq='6h')).agg(np.sum)
    EST6=DATA6.loc[np.in1d(DATA6.index.month, 2)].dropna().apply(estadisticosprec, axis=0)
    DATA12=month_sat_hourly_corrected.groupby(pd.Grouper(freq='12h')).agg(np.sum)
    EST12=DATA12.loc[np.in1d(DATA12.index.month, 2)].dropna().apply(estadisticosprec, axis=0)
    DATA18=month_sat_hourly_corrected.groupby(pd.Grouper(freq='18h')).agg(np.sum)
    EST18=DATA18.loc[np.in1d(DATA18.index.month, 2)].dropna().apply(estadisticosprec, axis=0)
    EST24=month_obs_daily.apply(estadisticosprec, axis=0)
    
    names=['stattion','x','y']
    agregados=[24,3,6,12,18]
    ESTS=[EST24,EST3,EST6,EST12,EST18]
    result=[]
    for i,stat in enumerate(ESTS):
        names=['x','y']
        names=names+['mean{}'.format(agregados[i]),'var{}'.format(agregados[i]),'autocov{}'.format(agregados[i]),'dryperiod{}'.format(agregados[i])]
        result.append(crearTablaEst(location,stat,names))

    df=pd.concat(result,axis=1)
    df=df.loc[:,~df.columns.duplicated()]
    return df

# OBS DAILY GAUGE STATIONS

In [ ]:
rng = pd.date_range('1981-01-01','2016-12-31', freq='d')
obs=pd.read_csv('D:/Proyectos_GitHub/TrmmCorrection/data/RAIN_GAUGE.csv')
obs_daily=obs.iloc[:,3:13152].transpose()
obs_daily=obs_daily.set_index(rng)

# JAN STATS

In [11]:
rng=pd.date_range('1998-01-01 00:00:00','2014-12-31 21:00:00', freq='3H')
jan_sat_hourly,posiciones,jan_hourly_date=csv_trmm(file='D:/Proyectos_GitHub/TrmmCorrection/data/TRMM_NATIONAL_ENE.csv',month=1,month_hourly_date=rng)
jan_sat_daily=jan_sat_hourly.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [16]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng=pd.date_range('1998-01-01','2014-12-31', freq='d')
jan_sat_daily,jan_obs_daily,daily_dates=common_period(rng,1,obs_daily,jan_sat_daily)

In [19]:
posiciones,jan_sat_daily,jan_obs_daily,jan_sat_hourly=filter_months_complete(0.8,jan_obs_daily,jan_sat_daily,posiciones,jan_sat_hourly)

In [20]:
jan_sat_hourly_corrected=bias_correction(jan_sat_daily,jan_obs_daily,jan_hourly_date,jan_sat_hourly,daily_dates)

In [21]:
stats_month(jan_sat_hourly,jan_obs_daily,posiciones)

,x,y,mean24,var24,autocov24,dryperiod24,mean3,var3,autocov3,dryperiod3,...,autocov6,dryperiod6,mean12,var12,autocov12,dryperiod12,mean18,var18,autocov18,dryperiod18
0,-80.53361,-3.633611,1.886072,69.095162,9.799845,0.719165,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,-80.45750,-3.813333,3.083231,109.391423,21.312226,0.561670,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,-80.45722,-3.508333,1.857495,67.992577,11.265585,0.791271,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-80.32194,-3.440556,2.362998,91.248403,9.480408,0.722960,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,-80.66083,-3.948889,2.042952,39.920727,8.426427,0.763810,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-69.99972,-17.559167,2.375750,19.102245,9.267064,0.554080,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
246,-69.77944,-17.525000,3.112429,20.841440,7.045815,0.440228,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
247,-69.62628,-17.579861,2.956632,21.040603,9.462752,0.513684,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
248,-69.46759,-17.393589,3.712238,25.935864,13.089671,0.449597,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


# FEB STATS

In [5]:
rng=pd.date_range('1999-02-01 00:00:00','2015-02-28 21:00:00', freq='3H')
feb_sat_hourly,posiciones,feb_hourly_date=csv_trmm(file='D:/Proyectos_GitHub/TrmmCorrection/data/TRMM_NATIONAL_FEB.csv',month=2,month_hourly_date=rng)
feb_sat_daily=feb_sat_hourly.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [6]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng=pd.date_range('1999-02-01','2015-02-28', freq='d')
feb_sat_daily,feb_obs_daily,daily_dates=common_period(rng,2,obs_daily,feb_sat_daily)

In [7]:
posiciones,feb_sat_daily,feb_obs_daily,feb_sat_hourly=filter_months_complete(0.8,feb_obs_daily,feb_sat_daily,posiciones,feb_sat_hourly)

In [8]:
feb_sat_hourly_corrected=bias_correction(feb_sat_daily,feb_obs_daily,feb_hourly_date,feb_sat_hourly,daily_dates)

In [9]:
stats_month(feb_sat_hourly,feb_obs_daily,posiciones)

,x,y,mean24,var24,autocov24,dryperiod24,mean3,var3,autocov3,dryperiod3,...,autocov6,dryperiod6,mean12,var12,autocov12,dryperiod12,mean18,var18,autocov18,dryperiod18
0,-80.53361,-3.633611,3.159336,77.801825,21.766635,0.590708,0.149008,1.304216,0.297407,0.953125,...,0.533993,0.922396,0.596033,6.873705,0.904858,0.861458,0.887714,11.323439,1.288208,0.811526
1,-80.45750,-3.813333,5.040156,154.176386,23.295429,0.483333,0.197731,1.292419,0.253930,0.898958,...,0.526782,0.842187,0.790924,6.798799,0.932035,0.742708,1.180616,10.742483,2.369339,0.655763
2,-80.45722,-3.508333,3.102296,105.140057,9.841639,0.684760,0.180979,1.621259,0.637415,0.937760,...,1.109950,0.893750,0.723917,9.059486,1.912192,0.819792,1.080121,13.366030,3.988602,0.750779
3,-80.32194,-3.440556,4.240023,126.956981,28.050210,0.629167,0.160892,1.398435,0.519164,0.944010,...,0.991622,0.909896,0.643567,7.023036,2.155358,0.847917,0.950582,11.196740,3.268523,0.799065
4,-80.66083,-3.948889,3.885179,84.833470,10.870223,0.648421,0.152080,1.332980,0.230275,0.934635,...,0.460925,0.888542,0.608321,7.131606,0.579450,0.807292,0.906614,10.869973,1.036485,0.750779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,-69.81306,-17.237222,3.991949,36.185657,15.454496,0.473418,0.167056,0.520128,0.062315,0.880469,...,0.062555,0.802604,0.668224,2.132558,0.076182,0.618750,0.975665,3.264133,0.478857,0.517134
252,-69.77944,-17.525000,3.571500,27.732492,8.831444,0.414583,0.140559,0.180199,0.015337,0.812760,...,0.013133,0.679167,0.562236,0.748436,0.017962,0.438542,0.829216,1.132701,0.220597,0.294393
253,-69.62628,-17.579861,3.776351,38.287256,14.104773,0.495495,0.146879,0.371701,0.037782,0.885417,...,0.042853,0.814583,0.587518,1.464957,0.115056,0.638542,0.871630,2.511166,0.416786,0.534268
254,-69.46759,-17.393589,4.105708,27.924056,13.166166,0.418750,0.183055,0.647259,0.066984,0.888802,...,0.081754,0.809375,0.732219,2.522312,0.353659,0.628125,1.084633,4.482084,0.473645,0.532710


# MAR STATS

In [22]:
rng=pd.date_range('1998-01-01 00:00:00','2014-12-31 21:00:00', freq='3H')
mar_sat_hourly,posiciones,mar_hourly_date=csv_trmm(file='D:/Proyectos_GitHub/TrmmCorrection/data/TRMM_NATIONAL_MAR.csv',month=3,month_hourly_date=rng)
mar_sat_daily=mar_sat_hourly.groupby(pd.Grouper(freq='24h')).agg(np.sum)

In [25]:
#todos los datos diarios dentro del rango de tiempo del TRMM
rng=pd.date_range('1998-01-01','2014-12-31', freq='d')
mar_sat_daily,mar_obs_daily,daily_dates=common_period(rng,3,obs_daily,mar_sat_daily)

In [26]:
posiciones,mar_sat_daily,mar_obs_daily,mar_sat_hourly=filter_months_complete(0.8,mar_obs_daily,mar_sat_daily,posiciones,mar_sat_hourly)

In [27]:
mar_sat_hourly_corrected=bias_correction(mar_sat_daily,mar_obs_daily,mar_hourly_date,mar_sat_hourly,daily_dates)

In [28]:
stats_month(mar_sat_hourly,mar_obs_daily,posiciones)

,x,y,mean24,var24,autocov24,dryperiod24,mean3,var3,autocov3,dryperiod3,...,autocov6,dryperiod6,mean12,var12,autocov12,dryperiod12,mean18,var18,autocov18,dryperiod18
0,-80.53361,-3.633611,3.324782,99.499053,11.435901,0.641366,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.015697,0.148582,-0.000247,0.998344
1,-80.45750,-3.813333,5.147083,129.898728,28.278972,0.449715,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.009147,0.013711,-0.000084,0.993377
2,-80.45722,-3.508333,3.615726,130.403503,11.391685,0.669355,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.017760,0.093593,-0.000316,0.995033
3,-80.32194,-3.440556,4.678790,206.815173,33.169194,0.631879,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.040661,0.592961,-0.001659,0.996689
4,-80.66083,-3.948889,4.797211,135.533924,26.367212,0.628083,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.004617,0.005816,-0.000021,0.993377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,-69.99972,-17.559167,1.707395,19.702034,6.388114,0.682510,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.005114,0.003319,-0.000026,0.990066
243,-69.77944,-17.525000,2.307343,17.156946,7.218648,0.555977,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.005114,0.003319,-0.000026,0.990066
244,-69.62628,-17.579861,1.815606,17.036430,4.054311,0.687885,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.008980,0.024807,-0.000081,0.995033
245,-69.46759,-17.393589,2.025746,15.566818,5.370052,0.633065,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.019197,0.084003,-0.000370,0.993377
